In [44]:
pip install --upgrade tensorflow


  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl (390.2 MB)
Using cached keras-3.7.0-py3-none-any.whl (1.2 MB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\intra\\Krish Naik projects\\ANN Classification\\venv\\Scripts\\tensorboard.exe' -> 'c:\\Users\\intra\\Krish Naik projects\\ANN Classification\\venv\\Scripts\\tensorboard.exe.deleteme'
Consider using the `--user` option or check the permissions.



In [45]:
import tensorflow as tensorflow
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [46]:
## load the ANN train model, scaler pickle, onehot encoding
model=load_model('model.h5')


## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
  onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
  label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
  scaler=pickle.load(file)



In [54]:
# example input data
input_data = {
  'CreditScore': 600,
  'Geography': 'France',
  'Gender': 'Male',
  'Age': 40,
  'Tenure': 3,
  'Balance': 60000,
  'NumOfProducts': 2,
  'HasCrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 50000
}

In [55]:
# one-hot encode 'Geography'
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\intra\Krish Naik projects\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [56]:
# convert the key-value pairs into data frame
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [57]:
## encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [58]:
# combine one-hot encoded columns with input data
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [59]:
## scaling input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [61]:
## predict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 34ms/step


array([[0.01834738]], dtype=float32)

In [62]:
prediction_proba=prediction[0][0]

In [63]:
prediction_proba

0.01834738

In [64]:
if prediction_proba>0.5:
  print('The customer is likely to churn.')
else:
  print('The customer is not likely to churn.')

The customer is not likely to churn.
